In [ ]:
# ================================================================
#   ADVANCED DATABASE PROJECT – DIABETES INDICATOR ANALYSIS
#   CSV vs SQL vs Mongita PERFORMANCE + BASIC ML CLASSIFICATION
# ================================================================

import pandas as pd
import numpy as np
import sqlite3
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix, classification_report
from sklearn import svm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import matplotlib.pyplot as plt

In [ ]:
# ================================================================
#  0. Download Dataset from Kaggle
# ================================================================

import kagglehub

# Download latest version
path = kagglehub.dataset_download("mohankrishnathalla/diabetes-health-indicators-dataset")

print("Path to dataset files:", path)

In [ ]:
# ================================================================
#  1. LOAD DATA (CSV)
# ================================================================

CSV_PATH = "diabetes_dataset.csv"
print("Loading CSV...")
df = pd.read_csv(CSV_PATH)
df.head()


In [ ]:
# ================================================================
# 2. BASIC STATISTICAL EXPLORATION
# ================================================================

print("\n=== BASIC STATS ===")
print(df.describe().T)

print("\n=== CLASS DISTRIBUTION ===")
print(df["diagnosed_diabetes"].value_counts(normalize=True))

# Visualize a few correlations
numeric_df = df.select_dtypes(include=[np.number])
plt.figure(figsize=(12,10))
plt.imshow(numeric_df.corr(), cmap='coolwarm')
plt.colorbar()
plt.title("Correlation Heatmap")
plt.show()




In [ ]:
# ================================================================
# 3. CSV QUERY TIMING
# ================================================================

def time_csv_query(query_func, name):
    start = time.perf_counter()
    result = query_func()
    end = time.perf_counter()
    print(f"{name} → {end-start:.6f} sec")
    return result

print("\n=== CSV QUERY PERFORMANCE ===")

q1 = time_csv_query(lambda: df[df["bmi"] > 30], "People with BMI > 30")
q2 = time_csv_query(lambda: df.groupby("age").size(), "Group by Age")
q3 = time_csv_query(lambda: df[(df["bmi"] > 25) & (df["systolic_bp"] > 130)], "Filter BMI > 25 & SystolicBP > 130")


In [ ]:
# ================================================================
# 4. LOAD CSV INTO SQLITE
# ================================================================

SQLITE_DB = "diabetes.db"

conn = sqlite3.connect(SQLITE_DB)
df.to_sql("diabetes", conn, if_exists="replace", index=False)

print("\nDatabase created:", SQLITE_DB)


In [ ]:
# ================================================================
# 5. SQL QUERY TIMING (NO INDEX)
# ================================================================

def sql_query(query, name):
    start = time.perf_counter()
    result = pd.read_sql_query(query, conn)
    end = time.perf_counter()
    print(f"{name} → {end-start:.6f} sec")
    return result

print("\n=== SQL QUERY PERFORMANCE (NO INDEX) ===")

# BMI > 30
sql_query("SELECT * FROM diabetes WHERE bmi > 30;", "SQL BMI > 30")

# Group by age
sql_query("SELECT age, COUNT(*) FROM diabetes GROUP BY age;", "SQL group by age")

# Derived HighBP condition (systolic ≥130 OR diastolic ≥80)
sql_query("""
    SELECT * 
    FROM diabetes 
    WHERE bmi > 25 
      AND (systolic_bp >= 130 OR diastolic_bp >= 80);
""", 
"SQL BMI > 25 & High BP Condition")



In [ ]:
# ================================================================
# 6. CREATE INDEXES + RE-TIME QUERIES
# ================================================================

print("\n=== CREATING INDEXES ===")

# Correct columns for THIS dataset
conn.execute("CREATE INDEX IF NOT EXISTS idx_bmi ON diabetes(bmi);")
conn.execute("CREATE INDEX IF NOT EXISTS idx_age ON diabetes(age);")

# Derived high blood pressure index:
# Use systolic_bp and diastolic_bp
conn.execute("CREATE INDEX IF NOT EXISTS idx_bp ON diabetes(systolic_bp, diastolic_bp);")

conn.commit()

print("\n=== SQL QUERY PERFORMANCE (WITH INDEX) ===")

# Query 1
sql_query("SELECT * FROM diabetes WHERE bmi > 30;", 
          "Indexed SQL bmi > 30")

# Query 2
sql_query("SELECT age, COUNT(*) FROM diabetes GROUP BY age;", 
          "Indexed SQL group by age")

# Query 3 (derived "HighBP")
sql_query("""
    SELECT * 
    FROM diabetes 
    WHERE bmi > 25
      AND (systolic_bp >= 130 OR diastolic_bp >= 80);
""",
"Indexed SQL bmi > 25 & High BP condition")

In [ ]:
# ================================================================
# 7. VIEW QUERY PLANS
# ================================================================

print("\n=== QUERY PLANS ===")

plans = [
    # Query 1: BMI > 30
    "EXPLAIN QUERY PLAN SELECT * FROM diabetes WHERE bmi > 30;",

    # Query 2: group by age
    "EXPLAIN QUERY PLAN SELECT age, COUNT(*) FROM diabetes GROUP BY age;",

    # Query 3: derived high blood pressure condition
    """
    EXPLAIN QUERY PLAN
    SELECT *
    FROM diabetes
    WHERE bmi > 25
      AND (systolic_bp >= 130 OR diastolic_bp >= 80);
    """
]

for p in plans:
    print("\n", p)
    print(pd.read_sql_query(p, conn))

In [ ]:
# ================================================================
# 8. MONGITA
# ================================================================
from mongita import MongitaClientDisk

client = MongitaClientDisk()
db = client['mongo_diabetes']  # Use your database name
collection = db['diabetes']  # Use your collection name


data_dict = df.head(100).to_dict('records')

collection.insert_many(data_dict)


def mongita_query(mongo_query, name):
    start = time.perf_counter()
    
    # Perform query
    result = list(collection.find(mongo_query))  # Get results as a list of documents
    
    end = time.perf_counter()
    print(f"{name} → {end-start:.6f} sec")


mongo_query_1 = {"bmi": {"$gt": 30}}
mongo_query_2 = {"$group": {"_id": "$age", "count": {"$sum": 1}}}  # Group by age
mongo_query_3 = {
    "bmi": {"$gt": 25},
    "$or": [
        {"systolic_bp": {"$gte": 130}},
        {"diastolic_bp": {"$gte": 80}}
    ]
}

mongita_query(mongo_query_1, "Mongita BMI: First Run")
mongita_query(mongo_query_2, "Mongita Group by Age: First Run")
mongita_query(mongo_query_3, "Mongita BMI > 25 & High BP: First Run")

collection.create_index([("bmi", 1)])  # Ascending index on 'bmi'
collection.create_index([("age", 1)])  # Ascending index on 'age'
# Mongita does not support compound indexs

print("")
mongita_query(mongo_query_1, "Mongita BMI: Indexed First Run")
mongita_query(mongo_query_2, "Mongita Group by Age: Indexed First Run")

curr = collection.find({})
svm_data = list(curr)


In [ ]:
# ================================================================
# 9. BASIC CLASSIFICATION (LOGISTIC REGRESSION)
# ================================================================

print("\n=== LOGISTIC REGRESSION MODEL ===")

# Correct target column for THIS dataset
target = "diagnosed_diabetes"

# Split features and target
X = df.drop(columns=[target])
y = df[target]

# Convert all categorical columns into numeric dummy variables
X = pd.get_dummies(X, drop_first=True)

# Standardize all numeric features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Logistic Regression model
model_lr = LogisticRegression(max_iter=500)   # increase max_iter for stability
model_lr.fit(X_train, y_train)

# Predictions
preds = model_lr.predict(X_test)

# Performance metrics
print("Accuracy:", accuracy_score(y_test, preds))
print("Precision:", precision_score(y_test, preds))
print("Recall:", recall_score(y_test, preds))


In [ ]:
# ================================================================
# 10. SIMPLE NEURAL NETWORK CLASSIFIER
# ================================================================

print("\n=== NEURAL NETWORK CLASSIFIER ===")

# The neural network uses the already preprocessed:
# X_train, X_test, y_train, y_test
# created in Section 8 after one-hot encoding + scaling.

nn = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # binary classification output
])

nn.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history = nn.fit(
    X_train, 
    y_train, 
    epochs=10, 
    validation_split=0.2, 
    verbose=1
)

loss, acc = nn.evaluate(X_test, y_test)
print(f"\nNN Accuracy: {acc:.4f}")

In [ ]:
# ================================================================
# 11. SVM
# ================================================================
X_test, y_test, y_train, y_test
model = svm.SVC(kernel='linear')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# ================================================================
# 12. FINAL NOTEBOOK SUMMARY
# ================================================================

print("\n=== PROJECT SUMMARY ===")
print("1. Loaded CSV (no indexing) and ran queries.")
print("2. Loaded data into SQLite.")
print("3. Timed SQL queries with and without indexes.")
print("4. Compared EXPLAIN QUERY PLAN results.")
print("5. Performed basic statistical analysis.")
print("6. Trained Logistic Regression classifier.")
print("7. Trained small Neural Network classifier.")
print("\nNotebook complete!")